to-do
- create a dataset of images and tag them as is_big_flare - true or false
  - run the current notebook to get the hourly images for 2015 (no consideration for flare/no-flare)
  - make this code a module to get a function that can pull desired images from SDO-dataset
    - function schema: 
      get_sdo_solar_images_from_aws(
          wanted_times, # list of datetimes for which images are desired
          save_folder_path, # folder path to save .png files
      ) -> fetched_image_paths # list of paths of image .png files saved
  - use the function to pull images pertaining to all big-flare events in 2015 and add to the images dataset
  - move on to making a flare classifier

In [1]:
# %matplotlib inline

import os
from typing import Union
import zarr

# import gcsfs
import s3fs
import sunpy.map

import dask.array as da
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import sunpy.visualization.colormaps as cm

from astropy.time import Time
from dask.distributed import Client, LocalCluster
from sunpy.visualization import axis_labels_from_ctype, wcsaxes_compat

from matplotlib import animation
from IPython.display import HTML

/Users/aishsk6/anaconda3/envs/mlproject/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting configs

matplotlib.use('Agg')

In [3]:
# AWS_ZARR_ROOT = (
#     "s3://gov-nasa-hdrl-data1/contrib/fdl-sdoml/fdl-sdoml-v2/sdomlv2_small.zarr/"
# )

AWS_ZARR_ROOT = (
    "s3://gov-nasa-hdrl-data1/contrib/fdl-sdoml/fdl-sdoml-v2/sdomlv2.zarr/2015/"
)

def s3_connection(path_to_zarr: os.path) -> s3fs.S3Map:
    """
    Instantiate connection to aws for a given path `path_to_zarr`
    """
    return s3fs.S3Map(
        root=path_to_zarr,
        s3=s3fs.S3FileSystem(anon=True),
        # anonymous access requires no credentials
        check=False,
    )


def load_single_aws_zarr(
    path_to_zarr: os.path,
    cache_max_single_size: int = None,
) -> Union[zarr.Array, zarr.Group]:
    """
    load zarr from s3 using LRU cache
    """
    return zarr.open(
        zarr.LRUStoreCache(
            store=s3_connection(path_to_zarr),
            max_size=cache_max_single_size,
        ),
        mode="r",
    )

## 2. Reading and loading the AIA data


The SDO ML dataset is stored in the Zarr format, a format for the storage of chunked, compressed, N-dimensional arrays with Numpy dtype. For an in-depth overview, see https://zarr.readthedocs.io/en/stable/tutorial.html.

In [4]:
# first, we create a group with the store data located on GCP.
root = load_single_aws_zarr(
    path_to_zarr=AWS_ZARR_ROOT,
)

In [5]:
# Using `root.tree()`, we are able to display the hierarchy (of `loc`).
# print(root.tree())

As shown in the tree, the heirachy consists of groups, each shown with their respective shape, and data type. In this example, we will primarily look at the 171 Å channel from 2010. This consists of 6135 512x512 images, stored as float32, and can be accessed as follows:

In [6]:
images_171a_zarray = root["171A"]

We could have alternatively accessed the 2010 data as:

```
loc = 'fdl-sdoml-v2/sdomlv2_small.zarr/2010'
```

which becomes increasingly useful in the full dataset (where the heirachy contains years 2010 - present).

**Loading with Dask**

We can then load this data into an array using dask.

In [7]:
all_image = da.from_array(images_171a_zarray)
all_image

dask.array<array, shape=(82806, 512, 512), dtype=float32, chunksize=(120, 512, 512), chunktype=numpy.ndarray>

As shown above, the data has the shape (6135, 512, 512), and is split into 52 chunks of (120, 512, 512), each of 125.83 MB; this is further visualised on the right. The data is now in a form to be manipulated like a Numpy array.

We can load and display one image now:

In [8]:
image=all_image[0,:,:]
plt.figure(figsize=(5,5))
colormap = plt.get_cmap('sdoaia171')
plt.imshow(image.compute(),origin='lower',vmin=10,vmax=1000,cmap=colormap)

Depending on the use-case, we may wish to extract a subset of this data in various ways. In the following sections we step through a number of potential operations that we may wish to make with the data.

### 2a. Selecting images based on header information

The new data includes all fits header information with the same keywords. To find out the AIA keyword definition, one can refer to the following online document:
http://jsoc.stanford.edu/~jsoc/keywords/AIA/AIA02840_K_AIA-SDO_FITS_Keyword_Document.pdf

And one can list all the AIA keywords included:

We can extract the exposure (and observation) time from the data attributes (the header information), and downsample our data based upon that information.

### 2b. Selecting images based on indices

While the data is not currently ordered by observation time, we can simple index the array to extract a number of observations

In [9]:
# We are going to choose data in 1 hour intervals


# df_time = pd.DataFrame(t_obs, index=np.arange(np.shape(t_obs)[0]), columns=["Time"])
# df_time["Time"] = pd.to_datetime(df_time["Time"])

# select times at a frequency of 60 minutes
wanted_times = pd.date_range(
    start="2015-01-01 00:00:00", end="2015-12-31 23:59:59", freq="60T", tz="UTC"
)

In [10]:
# [NEW] select subset of images based on time

# get the indices of the images that are closest to the wanted times: images_zry_wanted_idxs
images_zry_wanted_idxs = []
images_zry_times = pd.to_datetime(np.array(images_171a_zarray.attrs["T_OBS"]))
for selected_time in wanted_times[None:None]:
    images_zry_wanted_idxs.append(np.argmin(abs(images_zry_times - selected_time)))
images_zry_wanted_idxs = sorted(set(images_zry_wanted_idxs))

# get wanted times
images_wanted_times = images_zry_times[images_zry_wanted_idxs]

print("images_zry_wanted_idxs = ", images_zry_wanted_idxs)
print("len(images_zry_wanted_idxs) = ", len(images_zry_wanted_idxs))

images_zry_wanted_idxs =  [0, 10, 20, 30, 40, 50, 60, 61, 64, 73, 83, 93, 103, 113, 123, 133, 143, 153, 163, 173, 183, 192, 202, 212, 222, 232, 242, 252, 262, 272, 282, 292, 302, 312, 322, 332, 342, 352, 362, 372, 382, 392, 402, 412, 422, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 523, 525, 534, 544, 554, 564, 574, 584, 594, 604, 614, 624, 634, 644, 653, 663, 673, 683, 693, 703, 713, 723, 733, 743, 745, 753, 762, 772, 782, 792, 802, 812, 822, 832, 842, 852, 862, 872, 881, 891, 901, 911, 921, 931, 941, 951, 961, 971, 981, 991, 1001, 1011, 1021, 1031, 1041, 1051, 1061, 1071, 1081, 1091, 1101, 1111, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1299, 1300, 1304, 1314, 1324, 1334, 1343, 1353, 1363, 1373, 1383, 1393, 1403, 1413, 1423, 1433, 1436, 1439, 1448, 1458, 1468, 1478, 1488, 1498, 1508, 1518, 1528, 1538, 1547, 1556, 1565, 1575, 1585, 1595, 1605, 1615, 1625, 1635, 1645, 1655, 1656, 1661, 1671, 1681, 1691, 1701, 1711,

In [11]:
# get one image from aws


def get_single_solar_image(image_idx, path_to_zarr=AWS_ZARR_ROOT):
    images_drry = da.from_array(load_single_aws_zarr(path_to_zarr)["171A"])
    image = np.array(images_drry[image_idx, :, :])
    return image

# image = get_single_solar_image(images_zry_wanted_idxs[4002])

# # downsample the image to 256 by sampling every other pixel
# downsampled_pxl_posns = np.arange(0, image.shape[0], 2)
# image_downsmpd = image[downsampled_pxl_posns, :][:, downsampled_pxl_posns]

# plt.figure(figsize=(5, 5))
# plt.imshow(image, origin="lower", vmin=10, vmax=1000, cmap=plt.get_cmap("sdoaia171"))
# plt.figure(figsize=(5, 5))
# plt.imshow(
#     image_downsmpd, origin="lower", vmin=10, vmax=1000, cmap=plt.get_cmap("sdoaia171")
# )

In [12]:
# ASH is TESTINg.....

# TODO: change the code to track images_processed_times and don't use images_df
#       - also stop using csvs
#       - save each image as a PNG and name it with the datetime in format
#         dt=yyyy-mm-dd_hhmmss.png e.g. dt=2015-01-01_170400.png

import pandas as pd
import glob
import os
import re
import sys

# get the images that have been processed already: images_processed_times
images_png_folder = "/Users/aishsk6/gd_to_be_archived_big_files/sdo_image_data"
images_processed_paths = glob.glob(os.path.join(images_png_folder, "*.png"))
images_processed_times = [
    pd.to_datetime(re.sub(".png", "", os.path.basename(path)))
    for path in images_processed_paths
]

for image_time in images_wanted_times[None:None]:
    current_img_time = image_time
    # print(f"current_img_time: {current_img_time}")

    # get the position of image_time in images_wanted_times
    image_time_idx = list(images_wanted_times).index(image_time)
    # print(f"image_time_idx: {image_time_idx}")

    # check if the images_processed_times contains the row currently being processed and skip iter if true
    if current_img_time in images_processed_times:
        # print('current_img_time:', current_img_time, 'images_processed_times:', images_processed_times)
        print(
            f"Skipping image_time_idx {image_time_idx} as it has been processed already."
        )
        continue

    # get current image
    image_arr = get_single_solar_image(images_zry_wanted_idxs[image_time_idx])
    downsampled_pxl_posns = np.arange(0, image_arr.shape[0], 2)
    image_arr = image_arr[downsampled_pxl_posns, :][:, downsampled_pxl_posns]
    
    # Save the image
    fig = plt.figure(figsize=(5, 5))
    plt.imshow(image_arr, origin="lower", vmin=10, vmax=1000, cmap="sdoaia171")
    plt.savefig(f"{images_png_folder}/{current_img_time}.png")
    plt.close("all")  # Close the figure manually to release resources

    print(f"image_time_idx: {image_time_idx} of {len(images_wanted_times)}")

Skipping image_time_idx 0 as it has been processed already.
Skipping image_time_idx 1 as it has been processed already.
Skipping image_time_idx 2 as it has been processed already.
Skipping image_time_idx 3 as it has been processed already.
Skipping image_time_idx 4 as it has been processed already.
Skipping image_time_idx 5 as it has been processed already.
Skipping image_time_idx 6 as it has been processed already.
Skipping image_time_idx 7 as it has been processed already.
Skipping image_time_idx 8 as it has been processed already.
Skipping image_time_idx 9 as it has been processed already.
Skipping image_time_idx 10 as it has been processed already.
Skipping image_time_idx 11 as it has been processed already.
Skipping image_time_idx 12 as it has been processed already.
Skipping image_time_idx 13 as it has been processed already.
Skipping image_time_idx 14 as it has been processed already.
Skipping image_time_idx 15 as it has been processed already.
Skipping image_time_idx 16 as it h

In [ ]:
import sys

sys.getsizeof(images_zry_times) / (1024 ** 2)

In [ ]:
images_processed_times

In [ ]:
current_img_time